

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_ES.ipynb)




# **Detect entities in Spanish text**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

## 2. Start the Spark session

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 3. Select the DL model

In [4]:
# If you change the model, re-run all the cells below.
# Applicable models: wikiner_6B_100, wikiner_6B_300, wikiner_840B_300
MODEL_NAME = "wikiner_840B_300"

## 4. Some sample examples

In [5]:
# Enter examples to be transformed as strings in this list
text_list = ["""William Henry Gates III (nacido el 28 de octubre de 1955) es un magnate de los negocios, desarrollador de software, inversor y filántropo estadounidense. Es mejor conocido como el cofundador de Microsoft Corporation. Durante su carrera en Microsoft, Gates ocupó los cargos de presidente, director ejecutivo (CEO), presidente y arquitecto de software en jefe, y también fue el mayor accionista individual hasta mayo de 2014. Es uno de los empresarios y pioneros más conocidos de revolución de la microcomputadora de los años setenta y ochenta. Nacido y criado en Seattle, Washington, Gates cofundó Microsoft con su amigo de la infancia Paul Allen en 1975, en Albuquerque, Nuevo México; se convirtió en la compañía de software de computadora personal más grande del mundo. Gates dirigió la compañía como presidente y CEO hasta que dejó el cargo de CEO en enero de 2000, pero siguió siendo presidente y se convirtió en el arquitecto jefe de software. A fines de la década de 1990, Gates había sido criticado por sus tácticas comerciales, que se han considerado anticompetitivas. Esta opinión ha sido confirmada por numerosas sentencias judiciales. En junio de 2006, Gates anunció que haría la transición a un puesto de medio tiempo en Microsoft y trabajaría a tiempo completo en la Fundación Bill y Melinda Gates, la fundación caritativa privada que él y su esposa, Melinda Gates, establecieron en 2000. [ 9] Poco a poco transfirió sus deberes a Ray Ozzie y Craig Mundie. Renunció como presidente de Microsoft en febrero de 2014 y asumió un nuevo cargo como asesor tecnológico para apoyar al recién nombrado CEO Satya Nadella.""",
             """La Mona Lisa es una pintura al óleo del siglo XVI creada por Leonardo. Se celebra en el Louvre de París."""]

## 5. Define Spark NLP pipeline

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The wikiner_840B_300 is trained with glove_840B_300, so the embeddings in the
# pipeline should match
if MODEL_NAME == "wikiner_840B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', lang='xx') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")

ner_model = NerDLModel.pretrained(MODEL_NAME, 'es') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[
        document_assembler, 
        tokenizer,
        embeddings,
        ner_model,
        ner_converter])

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]
wikiner_840B_300 download started this may take some time.
Approximate size to download 14.2 MB
[OK!]


## 6. Run the pipeline

In [7]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlp_pipeline.fit(df).transform(df)

## 7. Visualize results

In [8]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)